# Using DALI with JAX

### Overview

This simple example shows how to train a neural network implementet in JAX with DALI pipelines. It builds on MNIST training example from JAX codebse that can be found [here](https://github.com/google/jax/blob/main/examples/mnist_classifier_fromscratch.py). This example is 

We will use MNIST in Caffe2 format from [DALI_extra](https://github.com/NVIDIA/DALI_extra)

In [31]:
import os

training_data_path = os.path.join(os.environ['DALI_EXTRA_PATH'], 'db/MNIST/training/')
validation_data_path = os.path.join(os.environ['DALI_EXTRA_PATH'], 'db/MNIST/testing/')

First step is to prepare function that will be later used to create instances of DALI pipelines. 

In [32]:
from nvidia.dali import pipeline_def
import nvidia.dali.fn as fn
import nvidia.dali.types as types


batch_size = 100
image_size = 28
num_classes = 10


@pipeline_def(device_id=0, batch_size=batch_size, num_threads=4)
def mnist_pipeline(data_path, random_shuffle):
    jpegs, labels = fn.readers.caffe2(
        path=data_path,
        random_shuffle=random_shuffle,
        name="mnist_caffe2_reader")
    images = fn.decoders.image(
        jpegs, device='mixed', output_type=types.GRAY)
    images = fn.crop_mirror_normalize(
        images, dtype=types.FLOAT, std=[255.], output_layout="CHW")
    images = fn.reshape(images, shape=[image_size * image_size])

    labels = labels.gpu()
    
    if random_shuffle:
        labels = fn.one_hot(labels, num_classes=num_classes)

    return images, labels

Next step is to instantiate pipelines and build them. Building creates and initializes pipeline internals.

In [33]:
print('Creating pipelines')
training_pipeline = mnist_pipeline(data_path=training_data_path, random_shuffle=True)
validation_pipeline = mnist_pipeline(data_path=validation_data_path, random_shuffle=False)

print('Building pipelines')
training_pipeline.build()
validation_pipeline.build()

print(training_pipeline)
print(validation_pipeline)

Creating pipelines
Building pipelines


This example focuses on how to use DALI pipeline with JAX. For more information on DALI pipeline look into [Getting started](../../getting_started.ipynb) and [pipeline documentation](../../../pipeline.rst)

For DALI pipeline to work with JAX it needs to be wrapped with appropriate DALI iterator. To get iterator compatible with JAX we need to import in from DALI JAX plugin. In addition to the pipeline we can pass the `output_map`, `reader_name` and `auto_reset` to the iterator. Quick explnation of how these parameters work:

 - `output_map`: iterators return a dictionary with outputs of the pipeline as its values. Keys in this dictionary are defined by `output_map`. For example, `labels` output returned from the DALI pipeline defined above will be accessible as `iterator_output['labels']`,
 - `reader_name`: setting this parameter introduces the notion of an epoch to our iterator. DALI pipeline itself is infinite, it will return the data indefinately, wrapping around the dataset. DALI readers (such as `fn.readers.caffe2` used in this example) have access to the information about the size of the dataset. If we want to pass this information to the iterator we need to point to the operator that should be queried for the dataset size. We do it by naming the operator (note `name="mnist_caffe2_reader"`) and passing the same name as the value for `reader_name` argument,
  - `auto_reset`: this argument controls the behaviour of the iterator after the end of an epoch. If set to `True` will automatically reset the state of the iterator and prepare it to start the next epoch.

In [34]:
from nvidia.dali.plugin import jax as dax


print('Creating iterators')
training_iterator = dax.DALIGenericIterator(
    training_pipeline,
    output_map=["images", "labels"],
    reader_name="mnist_caffe2_reader",
    auto_reset=True)

validation_iterator = dax.DALIGenericIterator(
    validation_pipeline,
    output_map=["images", "labels"],
    reader_name="mnist_caffe2_reader",
    auto_reset=True)

print(training_iterator)
print(f"Training iterator size = {training_iterator.size}")
print(f"Validation iterator size = {validation_iterator.size}")

Creating iterators
Training iterator size = 60000
Validation iterator size = 10000


With the setup above DALI iterators are ready for the training. 

Next we import training utilities implemented in JAX and set some parameters of the training.

In [35]:
from model import init_random_params, update, accuracy

layer_sizes = [784, 1024, 1024, 10]
param_scale = 0.1
step_size = 0.001
num_epochs = 10

At this point everything is ready to run the training.

In [36]:
print('Starting training')


params = init_random_params(param_scale, layer_sizes)

for epoch in range(num_epochs):
    for batch in training_iterator:
        params = update(params, batch, step_size)

    test_acc = accuracy(params, validation_iterator)
    print(f"Epoch {epoch} sec")
    print(f"Test set accuracy {test_acc}")

Starting training
Epoch 0 sec
Test set accuracy 0.7789000272750854
Epoch 1 sec
Test set accuracy 0.8401000499725342
Epoch 2 sec
Test set accuracy 0.8641000390052795
Epoch 3 sec
Test set accuracy 0.8773000240325928
Epoch 4 sec
Test set accuracy 0.8871000409126282
Epoch 5 sec
Test set accuracy 0.8957000374794006
Epoch 6 sec
Test set accuracy 0.9025000333786011
Epoch 7 sec
Test set accuracy 0.9083000421524048
Epoch 8 sec
Test set accuracy 0.9104000329971313
Epoch 9 sec
Test set accuracy 0.912600040435791
